In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

In [2]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [3]:
def parse_sink_bag(input: str) -> List[Tuple[str, int]]:
    num, bag = re.search(r'(\d+) ([\w ]+) bags?', input).groups()
    return bag, int(num)
 
def parse_bags(input: str) -> List[str]:
    source, sink = re.search(r'([a-z\s]+) bags contain ([\w\s,]+).', input).groups()
    if "no other bags" in sink:
        return source, dict()
    return source, dict([parse_sink_bag(bag) for bag in sink.split(", ")])

def find_num_bag_colors(rules: dict, target='shiny gold') -> int:

    @lru_cache(maxsize=None)
    def contains(bag: str, target: str) -> bool:
        contents = rules.get(bag, {})
        return (target in contents 
                or any(contains(inner, target) for inner in contents))

    return sum(contains(bag, target) for bag in rules)

In [4]:
string = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [5]:
test_rules = dict(read_data(string, parser=parse_bags, sep="\n", testing=True))
find_num_bag_colors(test_rules)

4

Part I  

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [6]:
real_rules = dict(read_data("input.txt", parser=parse_bags))
find_num_bag_colors(real_rules)

226

Part II

How many individual bags are required inside your single shiny gold bag?

In [7]:
def sum_num_bags(rules: dict, start='shiny gold') -> int:
    bag_sum = 0
    for bag, num in rules.get(start, {}).items():
        bag_sum += num + num * sum_num_bags(rules, bag)
    return bag_sum


In [8]:
sum_num_bags(real_rules)

9569